In [1]:
dtypes = {
    'comment_text':'str',
    'comment_id':'int',
    'page_id': 'str',
    'parent_id': 'str', 
    'post_id': 'str',
    'comment_reaction_count':'float',
    'comment_reply_count':'float',
    'comment_reaction1':'str',
    'comment_reaction2':'str',
    'comment_reaction3':'str',
    'comment_topfan':'float',
    'like':'float',
    'super_like':'float',
    'sup_like':'float',
    'angry':'float',
    'super_angry':'float',
    'sup_angry':'float',
    'love':'float',
    'super_love':'float',
    'sup_love':'float',
    'haha':'float',
    'super_haha':'float',
    'sup_haha':'float',
    'sad':'float',
    'super_sad':'float',
    'sup_sad':'float',
    'wow':'float',
    'super_wow':'float',
    'sup_wow':'float',
    'is_skoi':'float',
    'thread_level': 'float',
    'post_image': 'float',
    'post_video': 'float',
    'post_comment_count':'float',
    'post_share_count':'float',
    'post_reaction_count':'float',
    'post_like': 'float',
    'post_love': 'float',
    'post_haha': 'float', 
    'post_sad': 'float',
    'post_wow': 'float',
    'post_angry': 'float',
    'comment_image':'float',
    'comment_video':'float',
    'comment_year':'float',
    'comment_month': 'float',
    'comment_day': 'float',
    'comment_hour': 'float',
    'comment_minute': 'float',
    'comment_second': 'float',
    'comment_day_of_week': 'float',
    'post_year':'float',
    'post_month':'float',
    'post_day':'float',
    'post_hour': 'float',
    'post_minute': 'float',
    'post_second': 'float',
    'post_day_of_week': 'float',
    'comment_text_char_len': 'float',
    'comment_num_hashtag': 'float',
    'comment_avg_char_hashtag':'float',
    'comment_urls_len':'float',
    'comment_neg_emo_count':'float',
    'comment_neu_emo_count': 'float',
    'comment_pos_emo_count': 'float',
    'comment_author': 'str'   
}
context_features = [
    'comment_reaction_count','comment_reply_count', 'comment_reaction1','comment_reaction2','comment_reaction3',
    'comment_topfan','like','super_like','sup_like','angry','super_angry','sup_angry','love','super_love','sup_love',
    'haha','super_haha','sup_haha','sad','super_sad','sup_sad','wow','super_wow','sup_wow','is_skoi',
    'thread_level','post_image','post_video','post_comment_count','post_share_count','post_reaction_count',
    'post_like','post_love','post_haha','post_sad','post_wow','post_angry',
    'comment_image','comment_video','comment_year','comment_month','comment_day','comment_hour',
    'comment_minute','comment_second','comment_day_of_week','post_year','post_month',
    'post_day','post_hour','post_minute','post_second','post_day_of_week','comment_text_char_len',
    'comment_num_hashtag','comment_avg_char_hashtag','comment_urls_len','comment_neg_emo_count',
    'comment_neu_emo_count','comment_pos_emo_count']

content_features = ['comment_text']
labels = ['rude','figurative','offensive','dirty']


reaction_map = {
    'haha': 1,
    'like': 2,
    'angry': 3,
    'sad': 4,
    'wow': 5,
    'love': 6,
    '<NONE>': 7
}

In [2]:
import pandas as pd
import numpy as np
from pythainlp import word_tokenize
from tqdm import tqdm_notebook
import re
import emoji

#viz
from plotnine import *
import matplotlib.pyplot as plt
import seaborn as sns

from ast import literal_eval
from collections import Counter

#fastai
import fastai
from fastai.text import *
from fastai.callbacks import CSVLogger

#pythainlp
from pythainlp.ulmfit import *


import numpy as np
from sklearn.model_selection import train_test_split

from fastai.text import *
from fastai.callbacks import CSVLogger, SaveModelCallback
from pythainlp.ulmfit import *


from sklearn.metrics import f1_score

import time



from sklearn.metrics import confusion_matrix

In [3]:
def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def ungroup_emoji(toks):
    res = []
    for tok in toks:
        if emoji.emoji_count(tok) == len(tok):
            for char in tok:
                res.append(char)
        else:
            res.append(tok)
    return res

def process_text(text):
    #pre rules
    res = text.lower().strip()
    res = replace_url(res)
    res = replace_rep(res)
    
    #tokenize
    res = [word for word in word_tokenize(res, engine='ulmfit') if word and not re.search(pattern=r"\s+", string=word)]
    
    #post rules
    res = ungroup_emoji(res)
    
    return res

In [4]:
res1 = np.load("embeding_arr.npy")
emb_dict = {v:i for i, v in enumerate(res1[:, 0])}
res1 = res1[:,1:]

def embedding_df(df):
    for i in range(400):
        df['emb_' + str(i)] = None

    a = np.zeros((len(df),400))
    for i, v in enumerate(df.comment_id):
        a[i] = res1[emb_dict[v]]

    df.iloc[:,df.columns.get_loc('emb_0'):] = a
    return df

In [5]:
def pre_df(df3):
    df3.rude = df3.rude.apply(int)
    df3.figurative = df3.figurative.apply(int)
    df3.offensive = df3.offensive.apply(int)
    df3.dirty = df3.dirty.apply(int)
    return df3

In [3]:
Corpus = pd.read_csv('../../MapLabeledData/UpdateLabel/dirty_vs_non.csv', dtype=dtypes)
Unlabeled_Corpus = pd.read_csv('../../MapLabeledData/UpdateLabel/unlabeled_data_temp1.csv', dtype=dtypes)

In [6]:
Corpus[['rude','figurative','offensive','dirty']].sum()

rude          351.0
figurative     48.0
offensive      76.0
dirty         595.0
dtype: float64

In [8]:
Unlabeled_Corpus[['rude','figurative','offensive','dirty']].sum()

KeyError: "None of [Index(['rude', 'figurative', 'offensive', 'dirty'], dtype='object')] are in the [columns]"

In [7]:
#remove duplicate
label_id_set = set(Corpus['comment_id'])
unlabel_id_set = set(Unlabeled_Corpus['comment_id'])
duplicate_comment_id_set = label_id_set.intersection(unlabel_id_set)
Unlabeled_Corpus = Unlabeled_Corpus.loc[~Unlabeled_Corpus['comment_id'].isin(duplicate_comment_id_set)]
Unlabeled_Corpus.reset_index(inplace=True)


In [8]:
reaction_columns = [f'comment_reaction{i}' for i in range(1, 4)]
Corpus[reaction_columns] = Corpus[reaction_columns].replace(reaction_map)
Unlabeled_Corpus[reaction_columns] = Unlabeled_Corpus[reaction_columns].replace(reaction_map)

In [9]:
Corpus = pre_df(Corpus)
Train_X, Test_X = train_test_split(Corpus, train_size = 0.8)

In [10]:
Train_Y = Train_X[['rude','figurative','offensive','dirty']]
Test_Y = Test_X[['rude','figurative','offensive','dirty']]

In [12]:
df_embed_train = embedding_df(Train_X)
df_embed_test = embedding_df(Test_X)
df_embed_unlabel = embedding_df(Unlabeled_Corpus)
embedding_train=df_embed_train.iloc[:, df_embed_train.columns.get_loc('emb_0'):]
embedding_test=df_embed_test.iloc[:, df_embed_test.columns.get_loc('emb_0'):]
embedding_unlabel=df_embed_unlabel.iloc[:, df_embed_unlabel.columns.get_loc('emb_0'):]

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [16]:
from scipy.sparse import hstack, csr_matrix, vstack
train_feat_context = Train_X[context_features].values
test_feat_context = Test_X[context_features].values
unlabeled_feat_context = Unlabeled_Corpus[context_features].values

train_feat_content = embedding_train.values
test_feat_content = embedding_test.values
unlabeled_feat_content = embedding_unlabel.values

In [17]:
def createGeneralEvaluation(file):
    f = open(file, mode='w')
    f.write("time,amount_of_data,data_type,threshold,type_class,accuracy,precision,recall,fscore,support,auc_roc,auc_pr")
    f.write("\n")
    f.close()

In [18]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
def report_performance(file,time,modelType,predictions_class_train_c, predictions_class_test_c, type_class,threshold, predict_train_prob,predict_test_prob, train_label,train_feat):
    #Evaluation class (test data)
    print("Test data of "+modelType)
    data_type_test = "Test data of "+modelType
    threshold = str(threshold)
    type_class = str(type_class)
    accuracy_test = str(accuracy_score(predictions_class_test_c, Test_Y[type_class]))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_test_c, Test_Y[type_class])*100)
    print("Confusion Matrix: \n", confusion_matrix(Test_Y[type_class],predictions_class_test_c))
    precision,recall,fscore,support=score(Test_Y[type_class],predictions_class_test_c)
    precision_test = (format(precision[1]))
    recall_test = (format(recall[1]))
    fscore_test = (format(fscore[1]))
    support_test = (format(support[1]))
    auc_roc_test = str(roc_auc_score(Test_Y[type_class], predict_test_prob))
    print("PRF: \n", classification_report(Test_Y[type_class],predictions_class_test_c))
    print("Area under the curve ROC: ", roc_auc_score(Test_Y[type_class], predict_test_prob))
    lr_precision_test, lr_recall_test, _ = precision_recall_curve(Test_Y[type_class], predict_test_prob)
    auc_pr_test = str(auc(lr_recall_test, lr_precision_test))
    print("Area under the curve PR: ", auc(lr_recall_test, lr_precision_test))
    print("-----------------------------------------------------------\n")

    #Evaluation class (train data)
    print("Train data of "+modelType)
    data_type_train = "Train data of "+modelType
    accuracy_train = str(accuracy_score(predictions_class_train_c, train_label))
    print(type_class+" Accuracy score:", 
          accuracy_score(predictions_class_train_c, train_label)*100)
    print("Confusion Matrix: \n",confusion_matrix(train_label,predictions_class_train_c))
    precision,recall,fscore,support=score(train_label,predictions_class_train_c)
    precision_train = (format(precision[1]))
    recall_train  = (format(recall[1]))
    fscore_train  = (format(fscore[1]))
    support_train  = (format(support[1]))
    auc_roc_train  = str(roc_auc_score(train_label, predict_train_prob))
    print("PRF: \n",classification_report(train_label,predictions_class_train_c))
    print("Area under the curve ROC: ", roc_auc_score(train_label, predict_train_prob))
    lr_precision_train, lr_recall_train, _ = precision_recall_curve(train_label, predict_train_prob)
    auc_pr_train  = str(auc(lr_recall_train, lr_precision_train))
    print("Area under the curve PR: ", auc(lr_recall_train, lr_precision_train))
    print("___________________________________________________________________________________________________________\n")
    
    f = open(file, mode='a')
    f.write(str(time)+","+str(train_feat)+","+data_type_test+","+threshold+","+type_class+","+accuracy_test+","+precision_test+","+recall_test+","+fscore_test+","+support_test+","+auc_roc_test+","+auc_pr_test)
    f.write("\n")
    f.write(str(time)+","+str(train_feat)+","+data_type_train+","+threshold+","+type_class+","+accuracy_train+","+precision_train+","+recall_train+","+fscore_train+","+support_train+","+auc_roc_train+","+auc_pr_train)
    f.write("\n")
    
    f.close()

In [19]:
import sklearn.ensemble as ensemble
import numpy
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier
    
def train_and_label(train_X, train_y, test_X, test_y, u_X, file, type_class, time, dataType):
    model = RandomForestClassifier(n_estimators=20, n_jobs=20)
    model.fit(train_X, train_y)
    
    # eval
    predictions_test = model.predict_proba(test_X)
    predictions_train = model.predict_proba(train_X)
    for th in numpy.arange(0.1,1.1,0.1):
        predictions_class_Train = (predictions_train[:,1] >= th).astype('int')
        predictions_class_Test = (predictions_test[:,1] >= th).astype('int')

        report_performance(file,time,dataType,predictions_class_Train,predictions_class_Test, type_class,th, predictions_train[:,1], predictions_test[:,1],train_y,train_X.shape[0])

    # add new data
    
    predictions_unlabeled = model.predict_proba(u_X)
    certain_positives = np.argwhere(predictions_unlabeled[:, 1] > 0.9).reshape(-1)
    certain_negatives = np.argwhere(predictions_unlabeled[:, 1] < 0.1).reshape(-1)
    return certain_positives, certain_negatives

Using TensorFlow backend.


In [20]:
file = 'report_embedding_dirty_add5000pertime.csv'
createGeneralEvaluation(file)

In [21]:
from scipy.sparse import hstack, csr_matrix, vstack
label = 'dirty'
_train_y = Train_Y[label].values
_test_y = Test_Y[label].values
content_train_y = _train_y
context_train_y = _train_y

content_certain_set = set()
content_certain_labels = pd.DataFrame(index=Unlabeled_Corpus.index)
content_certain_labels[label] = -1

context_certain_set = set()
context_certain_labels = pd.DataFrame(index=Unlabeled_Corpus.index)
context_certain_labels[label] = -1

for i in range(1,11):
    time = i
    # certain data from content
    _cp, _cn = train_and_label(
        train_feat_content, content_train_y, 
        test_feat_content, _test_y,
        unlabeled_feat_content, file, label, time, "content")

    # certain data from context
    _xp, _xn = train_and_label(
        train_feat_context, context_train_y, 
        test_feat_context, _test_y,
        unlabeled_feat_context, file, label, time, "context")

    # Update certain data from content to context model
    content_certain_set.update(list(_cp))
    content_certain_set.update(list(_cn))
    content_certain_labels.loc[_cp, label] = 1
    content_certain_labels.loc[_cn, label] = 0

    _ccl = list(content_certain_set)
    start = 5000*(time-1)
    end = 5000*(time)
    _ccl = _ccl[start:end]
    _cc_X = unlabeled_feat_context[_ccl]
    _cc_y = content_certain_labels.loc[_ccl, label]

    train_feat_context = vstack([train_feat_context, _cc_X])
    context_train_y = np.concatenate([context_train_y, _cc_y.values], axis=0)
    
    print('current amount of data in context model'+str(train_feat_context.shape))
    
    # Update certain data from context to content model
    context_certain_set.update(list(_xp))
    context_certain_set.update(list(_xn))
    context_certain_labels.loc[_xp, label] = 1
    context_certain_labels.loc[_xn, label] = 0

    _xxl = list(context_certain_set)
    _xxl = _xxl[start:end]
    _xx_X = unlabeled_feat_content[_xxl]
    _xx_y = context_certain_labels.loc[_xxl, label]

    train_feat_content = vstack([train_feat_content, _xx_X])
    content_train_y = np.concatenate([content_train_y, _xx_y.values], axis=0)

    print('current amount of data in content model'+str(train_feat_content.shape))

Test data of content
dirty Accuracy score: 83.87451843698403
Confusion Matrix: 
 [[2978  532]
 [  54   70]]
PRF: 
               precision    recall  f1-score   support

           0       0.98      0.85      0.91      3510
           1       0.12      0.56      0.19       124

    accuracy                           0.84      3634
   macro avg       0.55      0.71      0.55      3634
weighted avg       0.95      0.84      0.89      3634

Area under the curve ROC:  0.7820891921698374
Area under the curve PR:  0.3279138597045532
-----------------------------------------------------------

Train data of content
dirty Accuracy score: 95.76195390436877
Confusion Matrix: 
 [[13448   616]
 [    0   471]]
PRF: 
               precision    recall  f1-score   support

           0       1.00      0.96      0.98     14064
           1       0.43      1.00      0.60       471

    accuracy                           0.96     14535
   macro avg       0.72      0.98      0.79     14535
weighted avg  

PRF: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.99     14064
           1       1.00      0.20      0.34       471

    accuracy                           0.97     14535
   macro avg       0.99      0.60      0.66     14535
weighted avg       0.97      0.97      0.97     14535

Area under the curve ROC:  0.9999962259274557
Area under the curve PR:  0.9998868574594444
___________________________________________________________________________________________________________

Test data of content
dirty Accuracy score: 96.61529994496422
Confusion Matrix: 
 [[3510    0]
 [ 123    1]]
PRF: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      3510
           1       1.00      0.01      0.02       124

    accuracy                           0.97      3634
   macro avg       0.98      0.50      0.50      3634
weighted avg       0.97      0.97      0.95      3634

Area under the curve ROC

Area under the curve ROC:  0.9999913196331481
Area under the curve PR:  0.999745360015414
___________________________________________________________________________________________________________

current amount of data in context model(19535, 60)
current amount of data in content model(19535, 400)
Test data of content
dirty Accuracy score: 88.66263070996148
Confusion Matrix: 
 [[3157  353]
 [  59   65]]
PRF: 
               precision    recall  f1-score   support

           0       0.98      0.90      0.94      3510
           1       0.16      0.52      0.24       124

    accuracy                           0.89      3634
   macro avg       0.57      0.71      0.59      3634
weighted avg       0.95      0.89      0.91      3634

Area under the curve ROC:  0.7594694880985204
Area under the curve PR:  0.33308803532936027
-----------------------------------------------------------

Train data of content
dirty Accuracy score: 96.8466854363962
Confusion Matrix: 
 [[18448   616]
 [    0

Confusion Matrix: 
 [[3510    0]
 [ 117    7]]
PRF: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      3510
           1       1.00      0.06      0.11       124

    accuracy                           0.97      3634
   macro avg       0.98      0.53      0.55      3634
weighted avg       0.97      0.97      0.95      3634

Area under the curve ROC:  0.7594694880985204
Area under the curve PR:  0.33308803532936027
-----------------------------------------------------------

Train data of content
dirty Accuracy score: 97.90632198617865
Confusion Matrix: 
 [[19064     0]
 [  409    62]]
PRF: 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99     19064
           1       1.00      0.13      0.23       471

    accuracy                           0.98     19535
   macro avg       0.99      0.57      0.61     19535
weighted avg       0.98      0.98      0.97     19535

Area under the curv

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PRF: 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     59064
           1       1.00      0.02      0.04       471

    accuracy                           0.99     59535
   macro avg       1.00      0.51      0.52     59535
weighted avg       0.99      0.99      0.99     59535

Area under the curve ROC:  0.9999997663479508
Area under the curve PR:  0.9999707037368124
___________________________________________________________________________________________________________

Test data of context
dirty Accuracy score: 90.83654375343974
Confusion Matrix: 
 [[3205  305]
 [  28   96]]
PRF: 
               precision    recall  f1-score   support

           0       0.99      0.91      0.95      3510
           1       0.24      0.77      0.37       124

    accuracy                           0.91      3634
   macro avg       0.62      0.84      0.66      3634
weighted avg       0.97      0.91      0.93      3634

Area under the curve ROC

PRF: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     59061
           1       1.00      0.45      0.62       474

    accuracy                           1.00     59535
   macro avg       1.00      0.72      0.81     59535
weighted avg       1.00      1.00      0.99     59535

Area under the curve ROC:  0.9999997142338068
Area under the curve PR:  0.9999645045138053
___________________________________________________________________________________________________________

Test data of context
dirty Accuracy score: 96.75288937809576
Confusion Matrix: 
 [[3510    0]
 [ 118    6]]
PRF: 
               precision    recall  f1-score   support

           0       0.97      1.00      0.98      3510
           1       1.00      0.05      0.09       124

    accuracy                           0.97      3634
   macro avg       0.98      0.52      0.54      3634
weighted avg       0.97      0.97      0.95      3634

Area under the curve ROC

/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/posoma/miniconda3/envs/posoma/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


PRF: 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     59061
           1       1.00      0.05      0.09       474

    accuracy                           0.99     59535
   macro avg       1.00      0.52      0.54     59535
weighted avg       0.99      0.99      0.99     59535

Area under the curve ROC:  0.9999997142338068
Area under the curve PR:  0.9999645045138053
___________________________________________________________________________________________________________

current amount of data in context model(64535, 60)
current amount of data in content model(64535, 400)
